In [1]:
import requests, csv 
import us
import pandas as pd
import time
from easymoney.money import EasyPeasy
import json

/Users/gabrielmelmed/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


# I. Downloading the variables table

i. The first step is to the dictionary table for the ACS's variables and save it to a csv.The aim is to create a 'relational database' with a separate tables for values.

In [3]:
# change !! into a space in the label col
vars_df['label'] = vars_df['label'].str.replace('!!', ' ')

In [4]:
vars_df = vars_df[['index', 'label']]
vars_df

,index,label
0,for,Census API FIPS 'for' clause
1,in,Census API FIPS 'in' clause
2,ucgid,Uniform Census Geography Identifier clause
3,S0804_C04_068E,Estimate Public transportation (excluding taxi...
4,S0503_C02_078E,Estimate Foreign born; Born in Europe Civilian...
...,...,...
18822,S2402_C02_035E,"Estimate Male Full-time, year-round civilian e..."
18823,S1002_C04_004E,Estimate 60 years and over Percent distributio...
18824,S0601_C02_009E,Estimate Native; born in state of residence To...
18825,S2411_C01_012E,Estimate Median earnings (dollars) Civilian em...


In [5]:
# save as a csv
vars_df.to_csv("census_vars.csv", index=False)

In [ ]:
# find the variables that contain the word "rent"
rent_vars = vars_df[vars_df['label'].str.contains('rent', case=False)]

rent_vars

In [ ]:
income_vars = vars_df[vars_df['label'].str.contains('income', case=False) & vars_df['label'].str.contains('mean', case=False)]
income_vars

,index,label
285,S0506_C01_103E,Estimate Total INCOME IN THE PAST 12 MONTHS (I...
295,S0506_C01_105E,Estimate Total INCOME IN THE PAST 12 MONTHS (I...
302,S0506_C01_107E,Estimate Total INCOME IN THE PAST 12 MONTHS (I...
419,S0502_C01_108E,Estimate Total INCOME IN THE PAST 12 MONTHS (I...
435,S0502_C01_106E,Estimate Total INCOME IN THE PAST 12 MONTHS (I...
...,...,...
17800,S0503_C03_099E,Estimate Foreign born; Born in Northern and We...
18316,S2601A_C04_102E,Estimate Noninstitutionalized group quarters p...
18332,S2601A_C04_101E,Estimate Noninstitutionalized group quarters p...
18739,S0506_C05_099E,Estimate Foreign born; Born in Caribbean INCOM...


In [15]:
# find the lablel for index B21004_001E
vars_df[vars_df['index'] == 'S1903_C03_015E']

,index,label
3253,S1903_C03_015E,Estimate Median income (dollars) FAMILIES Fami...


In [ ]:
api_key  = "9330018ae4a2bd20da22ced1281145d9405dc349" # replace with your API key

ii. Store the values we want in a list

In [ ]:
vars_of_interest = [
    "S1903_C03_015E", # 'Estimate Median income (dollars) FAMILIES Families'
    "S1903_C03_001E", # 'Estimate Mean income (dollars) HOUSEHOLD INCOME All households'
    
]

In [21]:
dataset = 'acs/acs5' # American Community Survey 5-year data
series = "B21004_001E" # Median Income


responses = [] 
for year in list(range(2010, 2020))+[2022]: # Loop through years, there's no data for 2020
    data_url = f'https://api.census.gov/data/{year}/{dataset}?get=NAME,{series}&for=county:*&in=state:*&key={api_key}'
    print(f'{year}: {data_url}')
    df = pd.read_json(data_url) # Read data from API into a pandas dataframe
    df.columns = df.iloc[0] # Set first row as column names
    df = df[1:] # Remove first row
    df["Year"] = year
    time.sleep(1) # Sleep for 1 second to avoid hitting the API too fast
    responses.append(df)

2010: https://api.census.gov/data/2010/acs/acs5?get=NAME,B21004_001E&for=county:*&in=state:*&key=9330018ae4a2bd20da22ced1281145d9405dc349
2011: https://api.census.gov/data/2011/acs/acs5?get=NAME,B21004_001E&for=county:*&in=state:*&key=9330018ae4a2bd20da22ced1281145d9405dc349
2012: https://api.census.gov/data/2012/acs/acs5?get=NAME,B21004_001E&for=county:*&in=state:*&key=9330018ae4a2bd20da22ced1281145d9405dc349
2013: https://api.census.gov/data/2013/acs/acs5?get=NAME,B21004_001E&for=county:*&in=state:*&key=9330018ae4a2bd20da22ced1281145d9405dc349
2014: https://api.census.gov/data/2014/acs/acs5?get=NAME,B21004_001E&for=county:*&in=state:*&key=9330018ae4a2bd20da22ced1281145d9405dc349
2015: https://api.census.gov/data/2015/acs/acs5?get=NAME,B21004_001E&for=county:*&in=state:*&key=9330018ae4a2bd20da22ced1281145d9405dc349
2016: https://api.census.gov/data/2016/acs/acs5?get=NAME,B21004_001E&for=county:*&in=state:*&key=9330018ae4a2bd20da22ced1281145d9405dc349
2017: https://api.census.gov/data/

In [17]:
url=f"https://api.census.gov/data/2022/acs/acs5?get=NAME,B21004_001E&for=county:*&in=state:*&key={api_key}"

response = requests.get(url)

d = response.json()

In [20]:
url

'https://api.census.gov/data/2022/acs/acs5?get=NAME,B21004_001E&for=county:*&in=state:*&key=9330018ae4a2bd20da22ced1281145d9405dc349'

In [ ]:
https://api.census.gov/data/2010/acs/acs5?get=NAME,B21004_001E&for=country:*&in=state:*&key=9330018ae4a2bd20da22ced1281145d9405dc349

### Population

Population is easier. We can just use static download links 

In [ ]:
county_pop_url = "//www2.census.gov/programs-surveys/popest/tables/2020-2023/counties/totals/co-est2023-pop.xlsx"

